# Test: WHISP Concurrent & Sequential Processing

Testing new concurrent and sequential stats processing functions with proper logging, progress tracking, and endpoint validation.

## Test Structure

- Concurrent processing (high-volume endpoint)
- Sequential processing (standard endpoint)
- Results comparison and validation

## Part 1: Setup

Initialize Earth Engine and configure logging

In [64]:
import ee

# Reset Earth Engine completely
ee.Reset()
print("✅ Earth Engine reset")

✅ Earth Engine reset


## Part 2: CONCURRENT PROCESSING (High-Volume Endpoint)

Test concurrent processing with the high-volume endpoint

In [ ]:
import ee

# Initialize and set high-volume endpoint
try:
    ee.Initialize()
    print("Earth Engine reset")
    ee.data.setDebuggingEnabled(False)
    print("Initialized with high-volume endpoint")
except Exception as e:
    try:
        ee.Authenticate()
        
        ee.Initialize(opt_url='https://earthengine-highvolume.googleapis.com')
        print("Earth Engine reset and initialized with HIGH-VOLUME endpoint for benchmarking")
        print("Authenticated and initialized with high-volume endpoint")
    except:
        print("Using HIGH-VOLUME endpoint")
        if "high-volume" not in str(e).lower():
            print("WARNING: Not using high-volume endpoint!")

Earth Engine reset
Earth Engine reset and initialized with HIGH-VOLUME endpoint for benchmarking
Authenticated and initialized with high-volume endpoint
Earth Engine reset and initialized with HIGH-VOLUME endpoint for benchmarking
Authenticated and initialized with high-volume endpoint


In [66]:
# Verify endpoint is high-volume
api_url = str(ee.data._cloud_api_base_url)
if 'highvolume' in api_url:
    print("✅ Using HIGH-VOLUME endpoint")
else:
    print("❌ WARNING: Not using high-volume endpoint!")

✅ Using HIGH-VOLUME endpoint


In [67]:
import openforis_whisp as whisp
import logging
from openforis_whisp.advanced_stats import (
    setup_concurrent_logger,
    validate_ee_endpoint,
    whisp_stats_geojson_to_df_concurrent,
    check_ee_endpoint,
)

print("✅ Imported advanced stats module")

✅ Imported advanced stats module


In [54]:
# Setup logging for concurrent processing
logger = setup_concurrent_logger(level=logging.INFO)
logger.info("Logging configured")

INFO: Logging configured


In [55]:
# Choose if want to include additional custom layers
USE_CUSTOM_BANDS = True # set to True if want to add extra ee data to whisp

In [56]:
# Choose if want to include additional custom layers
USE_CUSTOM_BANDS = True # set to True if want to add extra ee data to whisp
# =============================================================================
# CUSTOM BANDS SETUP (OPTIONAL) - runs only if USE_CUSTOM_BANDS = True above
# =============================================================================
if USE_CUSTOM_BANDS:

    # Step 1: Define custom Earth Engine images (binary values 0 or 1)
    custom_images = {
        'example_treecover': ee.Image(1),  # ee.Image("UMD/hansen/global_forest_change_2024_v1_12").select("treecover2000").gt(10).selfMask()
        'nXX_example_commodity': ee.Image.random(seed=1).gte(.5).reproject(crs='EPSG:4326', scale=10) # ee.ImageCollection("projects/forestdatapartnership/assets/cocoa/model_2025a").filter(ee.Filter.date('2020-01-01', '2021-01-01')).mosaic().gt(.8).selfMask()
        # add more images as needed (prefix 'nXX_' = iso2 code for national dataset)
    }

    # Step 2: Define metadata for each custom band (keys must match above)
    # Themes: 'treecover', 'commodities', 'disturbance_before', 'disturbance_after'
    # Timber themes: 'primary', 'naturally_reg_2020', 'planted_plantation_2020', etc.
    custom_bands_info = {
        'example_treecover': {
            'ISO2_code': "",          # Country code (empty = all countries)
            'theme': 'treecover',     # Risk theme
            'theme_timber': "",       # Timber theme (if applicable)
            'use_for_risk': 1,        # Include in risk calculations (1=yes, 0=no)
            'use_for_risk_timber': 0  # Include in timber risk (1=yes, 0=no)
        },
        'nXX_example_commodity': {
            'ISO2_code': "XX", 
            'theme': 'commodities', 
            'theme_timber': "",
            'use_for_risk': 1, 
            'use_for_risk_timber': 0
        }
        # add more band metadata as needed
    }

    # Step 3: Combine custom bands and extract names
    custom_ee_image = whisp.combine_custom_bands(custom_images, custom_bands_info)

    custom_bands = list(custom_bands_info.keys())


In [57]:
# Choose additional national datasets to include (currently three countries: 'co', 'ci', 'br').
base_iso2_codes = ['co', 'ci', 'br']

# automatically add any custom ISO2 codes from custom_bands_info if USE_CUSTOM_BANDS is True
iso2_codes_list = base_iso2_codes.copy()
if USE_CUSTOM_BANDS:
    iso2_codes_list += [code.lower() for code in {v.get('ISO2_code') for v in custom_bands_info.values()} if code and code.lower() not in iso2_codes_list]

In [58]:
import openforis_whisp as whisp

print("Imported concurrent stats module")

Imported concurrent stats module


In [59]:
num_polygons=1000  # Smaller dataset for testing
min_area_ha=10 
max_area_ha=10 
min_number_vert=10     
max_number_vert=10   

In [60]:
# Generate test data (or use your own GeoJSON)
import geopandas as gpd
import json
import tempfile
import os
import io
from contextlib import redirect_stdout

state_geom = (ee.FeatureCollection("projects/sat-io/open-datasets/FAO/GAUL/GAUL_2024_L1")
    .filter(ee.Filter.inList('gaul1_name', ['Amazonas', 'Mato Grosso', 'Rondônia', 'Pará'])))
bounds = state_geom.geometry().bounds()

# Suppress GeoJSON generation messages
with redirect_stdout(io.StringIO()):
    random_geojson = whisp.generate_test_polygons(
        bounds=bounds, 
        num_polygons=num_polygons,
        min_area_ha=min_area_ha, 
        max_area_ha=max_area_ha, 
        min_number_vert=min_number_vert,     
        max_number_vert=max_number_vert     
    )

# Save to temporary file
temp_fd, concurrent_geojson_path = tempfile.mkstemp(suffix='.geojson', text=True)
os.close(temp_fd)
with open(concurrent_geojson_path, 'w') as f:
    json.dump(random_geojson, f)

print(f"Generated test GeoJSON with {len(random_geojson['features'])} features")
print(f"   Saved to: {concurrent_geojson_path}")

[utils.py | generate_test_polygons() | l.378] INFO: Extracting bounds from Earth Engine Geometry...
[utils.py | generate_test_polygons() | l.391] INFO: Bounds: [-78.71, -18.04, -46.06, 6.20]
[utils.py | generate_test_polygons() | l.419] INFO: Generating 1000 test polygons with 10-10 vertices...
[utils.py | generate_test_polygons() | l.391] INFO: Bounds: [-78.71, -18.04, -46.06, 6.20]
[utils.py | generate_test_polygons() | l.419] INFO: Generating 1000 test polygons with 10-10 vertices...
[utils.py | generate_test_polygons() | l.433] INFO: Generated 250/1000 polygons (25%)...
[utils.py | generate_test_polygons() | l.433] INFO: Generated 250/1000 polygons (25%)...
[utils.py | generate_test_polygons() | l.433] INFO: Generated 500/1000 polygons (50%)...
[utils.py | generate_test_polygons() | l.433] INFO: Generated 500/1000 polygons (50%)...
[utils.py | generate_test_polygons() | l.433] INFO: Generated 750/1000 polygons (75%)...
[utils.py | generate_test_polygons() | l.433] INFO: Generated 7

In [61]:
# Helper function to validate GeoJSON file size
def validate_geojson_size(geojson_path, max_size_mb=10):
    """
    Check if GeoJSON file size is within acceptable limits.
    
    Args:
        geojson_path: Path to the GeoJSON file
        max_size_mb: Maximum allowed size in MB (default: 10)
    
    Returns:
        tuple: (is_valid, size_mb, message)
    """
    import os
    
    file_size_bytes = os.path.getsize(geojson_path)
    file_size_mb = file_size_bytes / (1024 * 1024)
    
    is_valid = file_size_mb <= max_size_mb
    
    if is_valid:
        message = f"GeoJSON size OK: {file_size_mb:.2f} MB (limit: {max_size_mb} MB)"
    else:
        message = f"GeoJSON TOO LARGE: {file_size_mb:.2f} MB (limit: {max_size_mb} MB)"
    
    return is_valid, file_size_mb, message

# Test the validation function
test_path = concurrent_geojson_path
msg = validate_geojson_size(test_path, max_size_mb=10)
print(msg[2])

GeoJSON size OK: 0.64 MB (limit: 10 MB)


In [62]:
# Create Whisp image with national codes
iso2_codes = ['br', 'co', 'ci']

# whisp_image = whisp.combine_datasets(national_codes=iso2_codes)
# band_names = whisp_image.bandNames().getInfo()
# print(f"Created Whisp image with {len(band_names)} bands")

In [63]:
# Test concurrent: GeoJSON → DataFrame with automatic formatting
print("\n" + "="*70)
print("TEST 1: Concurrent GeoJSON → DataFrame (Formatted)")
print("="*70 + "\n")

try:
    df_concurrent = whisp.whisp_formatted_stats_geojson_to_df_concurrent(
        input_geojson_filepath=concurrent_geojson_path,
        # whisp_image=whisp_image,
        # custom_bands=custom_bands if USE_CUSTOM_BANDS else None,
        national_codes=iso2_codes,
        batch_size=10,
        max_concurrent=20,
        validate_geometries=False,
        add_metadata_server=False,
        logger=logger,
        
    )
    
    print(f"\n✅ SUCCESS: Concurrent processing complete!")
    print(f"   Processed: {df_concurrent.shape[0]} features")
    print(f"   Output columns: {df_concurrent.shape[1]}")
    print(f"\n   First row sample:")
    print(df_concurrent.iloc[0, :8])
    
except Exception as e:
    print(f"❌ ERROR: {str(e)}")
    import traceback
    traceback.print_exc()


TEST 1: Concurrent GeoJSON → DataFrame (Formatted)

INFO: Loading GeoJSON: C:\Users\Arnell\AppData\Local\Temp\tmpylvfdwa1.geojson
INFO: Loaded 1,000 features
INFO: Loaded 1,000 features
INFO: Processing 1,000 features in 100 batches
INFO: Processing 1,000 features in 100 batches


2025-11-04 14:41:53,025 - INFO - Created 10 records
2025-11-04 14:41:53,048 - INFO - Created 10 records
2025-11-04 14:41:53,048 - INFO - Created 10 records


Reading GeoJSON file from: C:\Users\Arnell\AppData\Local\Temp\tmp9g8tx7fv.geojson


2025-11-04 14:41:53,112 - INFO - Created 10 records
2025-11-04 14:41:53,159 - INFO - Created 10 records
2025-11-04 14:41:53,159 - INFO - Created 10 records


Reading GeoJSON file from: C:\Users\Arnell\AppData\Local\Temp\tmpq89pn8ok.geojson
Reading GeoJSON file from: C:\Users\Arnell\AppData\Local\Temp\tmpmdjjol34.geojson


...

Reading GeoJSON file from: C:\Users\Arnell\AppData\Local\Temp\tmp6bz32m4j.geojson
Reading GeoJSON file from: C:\Users\Arnell\AppData\Local\Temp\tmpstqhy39k.geojson


2025-11-04 14:42:44,461 - WARNING - Connection pool is full, discarding connection: earthengine-highvolume.googleapis.com. Connection pool size: 10
2025-11-04 14:42:44,622 - WARNING - Connection pool is full, discarding connection: earthengine-highvolume.googleapis.com. Connection pool size: 10
2025-11-04 14:42:44,622 - WARNING - Connection pool is full, discarding connection: earthengine-highvolume.googleapis.com. Connection pool size: 10
2025-11-04 14:42:45,442 - WARNING - Connection pool is full, discarding connection: earthengine-highvolume.googleapis.com. Connection pool size: 10
2025-11-04 14:42:45,442 - WARNING - Connection pool is full, discarding connection: earthengine-highvolume.googleapis.com. Connection pool size: 10
2025-11-04 14:42:46,204 - WARNING - Connection pool is full, discarding connection: earthengine-highvolume.googleapis.com. Connection pool size: 10
2025-11-04 14:42:46,204 - WARNING - Connection pool is full, discarding connection: earthengine-highvolume.googl

INFO: Progress: 100/100 (100% complete)
INFO: Processing complete: 100/100 batches
INFO: Processing complete: 100/100 batches


C:\Users\Arnell\Documents\GitHub\whisp\src\openforis_whisp\advanced_stats.py:1026: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  combined = pd.concat(results, ignore_index=True)


INFO: Processed 1,000 features successfully
Using cached schema for national_codes: ['br', 'co', 'ci']
[logger.py | info() | l.23] INFO: Found 5 extra columns: ['actual_area_ha', 'actual_vertices', 'internal_id', 'requested_area_ha', 'requested_vertices']
[logger.py | warning() | l.26] WARNING: Missing expected schema columns: ['external_id']
[logger.py | info() | l.23] INFO: No extra columns found in DataFrame.
Using cached schema for national_codes: ['br', 'co', 'ci']
[logger.py | info() | l.23] INFO: Found 5 extra columns: ['actual_area_ha', 'actual_vertices', 'internal_id', 'requested_area_ha', 'requested_vertices']
[logger.py | warning() | l.26] WARNING: Missing expected schema columns: ['external_id']
[logger.py | info() | l.23] INFO: No extra columns found in DataFrame.
[logger.py | info() | l.23] INFO: custom_bands=None: Excluding all custom bands (strict mode)
INFO: Concurrent processing + formatting + validation complete
[logger.py | info() | l.23] INFO: custom_bands=None: Ex

In [41]:
df_concurrent

,plotId,external_id,Area,Geometry_type,Country,ProducerCountry,Admin_Level_1,Centroid_lon,Centroid_lat,Unit,...,nBR_MapBiomas_col9_palmoil_2020,nBR_MapBiomas_col9_pc_2020,nBR_INPE_TCamz_cer_annual_2020,nBR_MapBiomas_col9_soy_2020,nBR_MapBiomas_col9_annual_crops_2020,nBR_INPE_TCamz_pasture_2020,nBR_INPE_TCcer_pasture_2020,nBR_MapBiomas_col9_pasture_2020,nCI_Cocoa_bnetd,geo
0,1,NaN,8.902,Polygon,COL,CO,Meta,-72.609578,2.938842,ha,...,0.0,0.0,0.0,0.0,0.0,0.000,0.000,0.000,0.0,"{'type': 'Polygon', 'coordinates': [[[-72.6111..."
1,2,NaN,9.497,Polygon,GUF,GF,Cayenne,-53.409679,5.485465,ha,...,0.0,0.0,0.0,0.0,0.0,0.000,0.000,0.000,0.0,"{'type': 'Polygon', 'coordinates': [[[-53.4112..."
2,3,NaN,10.219,Polygon,BRA,BR,Pará,-53.385549,-1.670386,ha,...,0.0,0.0,0.0,0.0,0.0,0.011,0.000,0.000,0.0,"{'type': 'Polygon', 'coordinates': [[[-53.3872..."
3,4,NaN,8.489,Polygon,BRA,BR,Bahia,-46.280713,-11.612091,ha,...,0.0,0.0,0.0,0.0,0.0,0.000,0.000,0.000,0.0,"{'type': 'Polygon', 'coordinates': [[[-46.2823..."
4,5,NaN,8.987,Polygon,BRA,BR,Acre,-71.806918,-9.445431,ha,...,0.0,0.0,0.0,0.0,0.0,0.000,0.000,0.000,0.0,"{'type': 'Polygon', 'coordinates': [[[-71.8085..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,996,NaN,9.229,Polygon,BRA,BR,Pará,-55.518330,-4.549403,ha,...,0.0,0.0,0.0,0.0,0.0,0.000,0.000,0.000,0.0,"{'type': 'Polygon', 'coordinates': [[[-55.5199..."
996,997,NaN,8.707,Polygon,NaN,NaN,NaN,-77.712314,6.057306,ha,...,0.0,0.0,0.0,0.0,0.0,0.000,0.000,0.000,0.0,"{'type': 'Polygon', 'coordinates': [[[-77.7139..."
997,998,NaN,9.342,Polygon,VEN,VE,Amazonas,-64.843860,4.540215,ha,...,0.0,0.0,0.0,0.0,0.0,0.000,0.000,0.000,0.0,"{'type': 'Polygon', 'coordinates': [[[-64.8454..."
998,999,NaN,9.216,Polygon,NaN,NaN,NaN,-78.330012,2.786583,ha,...,0.0,0.0,0.0,0.0,0.0,0.000,0.000,0.000,0.0,"{'type': 'Polygon', 'coordinates': [[[-78.3316..."


## Part 3: SEQUENTIAL PROCESSING (For Comparison)

Test sequential (standard endpoint) processing as an alternative approach

### Part 3A: Switch to Standard Endpoint

Switch from high-volume to standard endpoint for sequential testing

In [42]:
import ee

# Reset Earth Engine completely
ee.Reset()
print("✅ Earth Engine reset")

✅ Earth Engine reset


In [43]:
# Earth Engine initialization with STANDARD endpoint
try:
    ee.Initialize(opt_url='https://earthengine.googleapis.com')
    print("✅ Initialized with standard endpoint")
except Exception:
    ee.Authenticate()
    ee.Initialize(opt_url='https://earthengine.googleapis.com')
    print("✅ Authenticated and initialized with standard endpoint")

✅ Initialized with standard endpoint


In [ ]:
# Generate fresh test data for sequential testing (avoid caching)
print("\n" + "="*80)
print("GENERATING TEST DATA FOR SEQUENTIAL")
print("="*80)

# Suppress GeoJSON generation messages
with redirect_stdout(io.StringIO()):
    random_geojson_sequential = whisp.generate_test_polygons(
        bounds=bounds, 
        num_polygons=num_polygons,
        min_area_ha=min_area_ha, 
        max_area_ha=max_area_ha, 
        min_number_vert=min_number_vert,     
        max_number_vert=max_number_vert     
    )

# Save to temporary file
temp_fd_sequential, sequential_geojson_path = tempfile.mkstemp(suffix='.geojson', text=True)
os.close(temp_fd_sequential)
with open(sequential_geojson_path, 'w') as f:
    json.dump(random_geojson_sequential, f)

print(f"✅ Generated test GeoJSON with {len(random_geojson_sequential['features'])} features")
print(f"   Saved to: {sequential_geojson_path}")


GENERATING TEST DATA
[utils.py | generate_test_polygons() | l.378] INFO: Extracting bounds from Earth Engine Geometry...
[utils.py | generate_test_polygons() | l.391] INFO: Bounds: [-78.71, -18.04, -46.06, 6.20]
[utils.py | generate_test_polygons() | l.419] INFO: Generating 1000 test polygons with 10-10 vertices...
[utils.py | generate_test_polygons() | l.433] INFO: Generated 250/1000 polygons (25%)...
[utils.py | generate_test_polygons() | l.433] INFO: Generated 500/1000 polygons (50%)...
[utils.py | generate_test_polygons() | l.433] INFO: Generated 750/1000 polygons (75%)...
[utils.py | generate_test_polygons() | l.467] INFO: Generated 1000 polygons!
[utils.py | generate_test_polygons() | l.473] INFO: Vertex count - Requested: 10-10, Actual: 10-10
[utils.py | generate_test_polygons() | l.481] INFO: Area (ha) - Requested: 10.0-10.0, Actual: 7.8-10.9
Generated test GeoJSON with 1000 features
   Saved to: C:\Users\Arnell\AppData\Local\Temp\tmp8vutfu1y.geojson


TypeError: string indices must be integers, not 'str'

In [45]:
# Test sequential: GeoJSON → DataFrame (Sequential Processing)
print("\nTEST 2: Sequential GeoJSON → DataFrame (Sequential)")
print("-" * 80)

try:
    df_sequential = whisp.whisp_formatted_stats_geojson_to_df_sequential(
        input_geojson_filepath=sequential_geojson_path,
        national_codes=['BR'],
        add_metadata_client_side=True,
        logger=logger,
    )
    
    print(f"\n✅ SUCCESS: Sequential processing complete!")
    print(f"   Processed: {df_sequential.shape[0]} features")
    print(f"   Output columns: {df_sequential.shape[1]}")
    print("\n   First row preview:")
    print(df_sequential.iloc[0, :8])
    
except Exception as e:
    print(f"\n❌ FAILED: {e}")
    import traceback
    traceback.print_exc()
    df_sequential = None


TEST 2: Sequential GeoJSON → DataFrame (Sequential)
--------------------------------------------------------------------------------

❌ FAILED: name 'sequential_geojson_path' is not defined


Traceback (most recent call last):
  File "C:\Users\Arnell\AppData\Local\Temp\ipykernel_6300\925425433.py", line 7, in <module>
    input_geojson_filepath=sequential_geojson_path,
                           ^^^^^^^^^^^^^^^^^^^^^^^
NameError: name 'sequential_geojson_path' is not defined


In [ ]:
# Compare concurrent vs sequential results
print("\nCOMPARISON: Concurrent vs Sequential")
print("=" * 80)

if df_concurrent is not None and df_sequential is not None:
    print(f"\nConcurrent shape:  {df_concurrent.shape}")
    print(f"Sequential shape:  {df_sequential.shape}")
    
    # Verify they return same columns
    if set(df_concurrent.columns) == set(df_sequential.columns):
        print("\n✅ Column names match!")
    else:
        print("\n⚠️  Column names differ")
        print(f"   Concurrent columns: {set(df_concurrent.columns) - set(df_sequential.columns)}")
        print(f"   Sequential columns: {set(df_sequential.columns) - set(df_concurrent.columns)}")
    
    print(f"\n✅ Sequential is simpler and better for debugging")

In [ ]:
df_sequential